In [1]:
# Gerekli kütüphaneler import ediliyor
import os  # Dosya ve klasör işlemleri için
import json  # JSON formatındaki verileri okuyup yazmak için
import cv2  # OpenCV: Görüntü işleme işlemleri için
import numpy as np  # Sayısal işlemler ve dizi işlemleri için
import torch  # PyTorch: Derin öğrenme işlemleri için
import torch.nn as nn  # Yapay sinir ağı katmanları için
import torch.optim as optim  # Model eğitimi için optimizasyon algoritmaları
from torch.utils.data import DataLoader, Dataset  # Veri yükleyici ve özel veri seti sınıfı için
import typing  # Tür tanımlamaları için (örn. Tuple[int, int])
import onnxruntime as ort  # ONNX formatındaki modelleri çalıştırmak için
import speech_recognition as sr  # Sesli konuşmayı yazıya çeviren kütüphane

# Anime stiline dönüştürme sınıfı tanımlanıyor
class AnimeGAN:
    def __init__(self, model_path: str = '', downsize_ratio: float = 1.0):  # Yapıcı metod, model yolu ve boyut oranı alır
        self.model = ort.InferenceSession(model_path)  # ONNX model dosyası yüklenir
        self.downsize_ratio = downsize_ratio  # Boyut küçültme oranı kaydedilir

    def to_32s(self, x):  # 32’nin katı en yakın değeri döndüren fonksiyon
        return 256 if x < 256 else x - x % 32  # 32’nin katı boyut ayarı yapılır

    def process_frame(self, frame: np.ndarray, x32: bool = True) -> np.ndarray:  # Görseli modele uygun hale getirir
        h, w = frame.shape[:2]  # Yükseklik ve genişlik alınır
        if x32:
            h, w = self.to_32s(h), self.to_32s(w)  # Boyutlar 32’nin katı yapılır
        frame = cv2.resize(frame, (w, h))  # Görüntü yeniden boyutlandırılır
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB).astype(np.float32) / 127.5 - 1.0  # RGB’ye çevirilip normalize edilir
        img = np.transpose(img, (2, 0, 1))  # Kanal boyutu en başa alınır (HWC -> CHW)
        img = np.expand_dims(img, axis=0)  # Batch boyutu eklenir
        return img  # İşlenmiş tensör döndürülür

    def post_process(self, frame: np.ndarray, wh: typing.Tuple[int, int]) -> np.ndarray:  # Model çıktısını işleyip orijinal boyuta getirir
        frame = np.squeeze(frame)  # Gereksiz boyut çıkarılır (1,3,H,W -> 3,H,W)
        frame = np.transpose(frame, (1, 2, 0))  # Kanal en sona alınır (CHW -> HWC)
        frame = ((frame + 1.0) * 127.5).clip(0, 255).astype(np.uint8)  # Normalize geri alınır ve uint8 yapılır
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # BGR formatına çevrilir
        return cv2.resize(frame, wh)  # Orijinal boyuta döndürülür

    def __call__(self, frame: np.ndarray) -> np.ndarray:  # Sınıf çağrıldığında otomatik olarak stilize işlemini yapar
        h, w = frame.shape[:2]  # Görsel boyutları alınır
        input_tensor = self.process_frame(frame)  # Giriş tensörü hazırlanır
        output = self.model.run(None, {'input': input_tensor})[0]  # Model tahmini alınır
        return self.post_process(output, (w, h))  # Çıktı işlenip döndürülür

# İşaret dili veri seti sınıfı tanımlanıyor
class SignLanguageDataset(Dataset):  # PyTorch Dataset sınıfından türetiliyor
    def __init__(self, json_path, image_folder, image_size=(64, 64)):  # Yapıcı metod
        with open(json_path, 'r') as f:  # JSON dosyası açılır
            self.data = json.load(f)  # JSON verisi yüklenir
        self.image_paths = []  # Görsel yolları tutulacak
        self.labels = []  # Etiketler tutulacak
        self.label_map = {word: idx for idx, word in enumerate(self.data.keys())}  # Kelime -> sayı eşleşmesi
        self.image_folder = image_folder  # Görsel klasörü kaydedilir
        self.image_size = image_size  # Görsel boyutu kaydedilir

        for word, filenames in self.data.items():  # Tüm kelime ve görseller dolaşılır
            for filename in filenames:
                self.image_paths.append(os.path.join(image_folder, filename))  # Görsel yolu eklenir
                self.labels.append(self.label_map[word])  # Etiketi eklenir

    def __len__(self):  # Veri seti uzunluğu
        return len(self.image_paths)

    def __getitem__(self, idx):  # Belirli bir indeks için veri döndürülür
        image_path = self.image_paths[idx]  # Görsel yolu alınır
        image = cv2.imread(image_path)  # Görsel okunur
        image = cv2.resize(image, self.image_size)  # Yeniden boyutlandırılır
        image = image.transpose((2, 0, 1))  # Kanal en başa alınır (CHW)
        image = torch.tensor(image, dtype=torch.float32) / 255.0  # Normalize edilip tensöre çevrilir
        label = torch.tensor(self.labels[idx], dtype=torch.long)  # Etiket tensörü alınır
        return image, label  # Görsel ve etiketi döndürülür

# Basit bir yapay sinir ağı (ANN) tanımlanıyor
class ANNModel(nn.Module):  # PyTorch sinir ağı modülünden türetiliyor
    def __init__(self, num_classes):  # Yapıcı metod, sınıf sayısı alır
        super(ANNModel, self).__init__()  # Üst sınıf yapıcısı çağrılır
        self.flatten = nn.Flatten()  # 3D tensörü tek boyuta indirir
        self.fc1 = nn.Linear(64 * 64 * 3, 256)  # Girişten 256 nörona
        self.relu1 = nn.ReLU()  # Aktivasyon fonksiyonu
        self.fc2 = nn.Linear(256, 128)  # Orta katman
        self.relu2 = nn.ReLU()  # Aktivasyon
        self.fc3 = nn.Linear(128, num_classes)  # Çıkış katmanı

    def forward(self, x):  # İleri besleme işlemi
        x = self.flatten(x)  # Görseli düzleştir
        x = self.relu1(self.fc1(x))  # İlk katman
        x = self.relu2(self.fc2(x))  # İkinci katman
        x = self.fc3(x)  # Çıkış katmanı
        return x  # Tahmin döndürülür



In [ ]:
import cv2
import os
import json
import numpy as np

# AnimeGAN sınıfı tanımlı olmalı (önceden eklendiği gibi)
onnx_model_path = r'animegan.onnx'
animegan = AnimeGAN(model_path=onnx_model_path)  # AnimeGAN modeli yüklenir

# === JSON VERİSİ YÜKLE ===
with open("output_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)  # Bu satır eksikse hiçbir şey işlenmez!

# === İŞLEME FONKSİYONU ===
def preprocess_dataset(data, image_folder, output_folder="processed_images"):
    os.makedirs(output_folder, exist_ok=True)
    processed_data = {}

    for word, frames in data.items():
        processed_paths = []
        print(f"📦 Kelime işleniyor: {word}")
        for frame_info in frames:
            img_path = os.path.join(image_folder, os.path.basename(frame_info["image_path"]))
            print(f"📂 Dosya kontrol ediliyor: {img_path} - Var mı? {os.path.exists(img_path)}")
            if os.path.exists(img_path):
                img = cv2.imread(img_path)
                img = cv2.resize(img, (256, 256))
                anime_img = animegan(img)
                print("✨ AnimeGan filtresi uygulandı")
                save_path = os.path.join(output_folder, f"{word}_{len(processed_paths)}.jpg")
                cv2.imwrite(save_path, anime_img)
                processed_paths.append(save_path)

        processed_data[word] = processed_paths

    with open("processed_data.json", "w", encoding="utf-8") as f:
        json.dump(processed_data, f, ensure_ascii=False, indent=4)

    print("✅ Görseller başarıyla işlendi ve 'processed_data.json' dosyasına kaydedildi.")

# === FONKSİYON ÇAĞRISI ===
preprocess_dataset(data, "OutputImages")


In [2]:
import threading
import cv2
import time
import json
import speech_recognition as sr
import numpy as np

# === Türkçe Karakter Normalizasyon Fonksiyonu ===
def normalize_text(text):
    replacements = {
        "ç": "c", "ğ": "g", "ı": "i", "ö": "o", "ş": "s", "ü": "u",
        "Ç": "C", "Ğ": "G", "İ": "I", "Ö": "O", "Ş": "S", "Ü": "U"
    }
    for turkish_char, ascii_char in replacements.items():
        text = text.replace(turkish_char, ascii_char)
    return text

# JSON'dan işlenmiş veri yükleme
with open("processed_data.json", "r") as f:
    processed_data = json.load(f)

# Görüntüleri baştan belleğe al
cached_frames = {}
for word, paths in processed_data.items():
    cached_frames[word] = [cv2.imread(p) for p in paths]

def real_time_sign_language():
    words_queue = []
    stop_listening = threading.Event()

    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Kamera açılamadı.")
        return

    # Pencere ayarları
    screen_width = 3000
    screen_height = 1200
    cv2.namedWindow("İşaret Dili Animasyonu", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("İşaret Dili Animasyonu", screen_width, screen_height)
    cv2.moveWindow("İşaret Dili Animasyonu", 0, 0)

    def listen_microphone():
        recognizer = sr.Recognizer()
        mic = sr.Microphone()

        with mic as source:
            recognizer.adjust_for_ambient_noise(source)

        while not stop_listening.is_set():
            with mic as source:
                try:
                    print(" Konuşabilirsiniz...")
                    audio = recognizer.listen(source, timeout=3, phrase_time_limit=5)
                    text = recognizer.recognize_google(audio, language="tr-TR")
                    words = text.split()
                    if words:
                        normalized_words = [normalize_text(w).capitalize() for w in words]
                        words_queue.extend(normalized_words)
                        print(f" Algılandı: {' '.join(normalized_words)}")
                except sr.UnknownValueError:
                    print(" Anlaşılamadı, lütfen tekrar edin...")
                except sr.RequestError:
                    print(" API hatası, internet bağlantınızı kontrol edin.")
                except:
                    continue

    threading.Thread(target=listen_microphone, daemon=True).start()

    print(" Kamera başlatıldı. Konuştuğunuz kelimelerin işaret dilini gösterecek. Çıkmak için 'q' tuşuna basın...")

    current_word = ""
    frame_list = []
    last_word = ""
    word_start_time = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Aynalama kaldırıldı (flip yok)
        user_frame = cv2.resize(frame, (screen_width, screen_height))
        anime_frame = np.zeros((500, 500, 3), dtype=np.uint8)

        # Yeni kelime kuyruğu varsa ve animasyon yoksa
        if words_queue and not frame_list:
            current_word = words_queue.pop(0)
            if current_word in cached_frames:
                frame_list = cached_frames[current_word]
                word_start_time = time.time()
                last_word = current_word  # Son kelimeyi güncelle
            else:
                print(f" Veri bulunamadı: {current_word}")
                frame_list = []

        # Animasyon varsa zamanla göster
        if frame_list:
            frame_count = len(frame_list)
            elapsed_time = (time.time() - word_start_time) * 75
            index = int(elapsed_time)

            if index < frame_count:
                anime_frame = cv2.resize(frame_list[index], (500, 500))
            else:
                frame_list = []
        elif last_word in cached_frames:
            # Eğer yeni kelime algılanmadıysa, son kelimenin ilk karesi gösterilsin
            anime_frame = cv2.resize(cached_frames[last_word][0], (500, 500))

        # Anime frame'i sağ alt köşeye yerleştir
        x_offset = screen_width - 500 - 20
        y_offset = screen_height - 500 - 20
        user_frame[y_offset:y_offset + 500, x_offset:x_offset + 500] = anime_frame

        cv2.imshow("İşaret Dili Animasyonu", user_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            stop_listening.set()
            break

    cap.release()
    cv2.destroyAllWindows()
    print(" Sistem durduruldu.")


In [3]:
# Canlı sistem başlat
real_time_sign_language()

 Kamera başlatıldı. Konuştuğunuz kelimelerin işaret dilini gösterecek. Çıkmak için 'q' tuşuna basın...
 Konuşabilirsiniz...
 Algılandı: Merhaba
 Konuşabilirsiniz...
 Konuşabilirsiniz...
 Algılandı: Ben
 Konuşabilirsiniz...
 Algılandı: Universite
 Konuşabilirsiniz...
 Anlaşılamadı, lütfen tekrar edin...
 Konuşabilirsiniz...
 Algılandı: Ogrencisiyim
 Konuşabilirsiniz...
 Algılandı: Simdi Proje Gelistiriyorum
 Konuşabilirsiniz...
 Anlaşılamadı, lütfen tekrar edin...
 Konuşabilirsiniz...
 Konuşabilirsiniz...
 Konuşabilirsiniz...
 Konuşabilirsiniz...
 Konuşabilirsiniz...
 Algılandı: Uzmanlar Uyarildi
 Konuşabilirsiniz...
 Konuşabilirsiniz...
 Sistem durduruldu.


 Anlaşılamadı, lütfen tekrar edin...
